In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import pandas as pd
import numpy as np
import json
import keras
import tensorflow as tf
import io
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.layers import Conv1D, Conv2D
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import LabelEncoder
import nltk
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.layers import average
import tensorflow_hub as hub
from keras.layers import Average
from keras.layers import Concatenate
nltk.download('punkt')
from numpy import random
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import gc
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
#plt.switch_backend('agg')
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
!pip install keras_metrics
import keras_metrics as km

In [0]:
root_path='/content/drive/My Drive/offenseval/2020'

In [0]:
train_data=pd.read_csv(root_path+'/non_english_data/Danish/offenseval-da-training-v1.tsv',delimiter='\t')

In [0]:
print(train_data.head())

     id                                              tweet subtask_a
0  3131  Jeg tror det vil være dejlig køligt, men jeg v...       NOT
1   711  Så kommer de nok til at investere i en ny cyke...       NOT
2  2500  Nu er det jo også de Ikea-aber der har lavet s...       OFF
3  2678  128 Varme emails, er vi enige om at det er sex...       NOT
4   784  Desværre tyder det på, at amerikanerne er helt...       NOT


## Tokenization and Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.model_selection import StratifiedKFold

In [0]:
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

In [0]:
df_train,df_dev=train_test_split(train_data,test_size=0.1)

In [0]:
y_train=(df_train['subtask_a']=="OFF").astype(int)
y_dev=(df_dev['subtask_a']=="OFF").astype(int)

In [0]:
print(train_data.shape)

(2961, 3)


In [0]:
max_words =3000
max_len = 45
tok = Tokenizer(max_words)
tok.fit_on_texts(train_data['tweet'].astype(str))

In [0]:
# sequences_train = tok.texts_to_sequences(df_train['tweet'].astype(str))
# vocab_size = len(tok.word_index) + 1
# sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [0]:
# sequences_dev = tok.texts_to_sequences(df_dev['tweet'].astype(str))
# vocab_size = len(tok.word_index) + 1
# sequences_matrix_dev = sequence.pad_sequences(sequences_dev,maxlen=max_len,padding='post',truncating='post')

In [0]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})

## Model

In [0]:
word_index=tok.word_index

In [0]:
print(word_index)

{'er': 1, 'det': 2, 'at': 3, 'og': 4, 'i': 5, 'en': 6, 'jeg': 7, 'ikke': 8, 'på': 9, 'de': 10, 'der': 11, 'til': 12, 'for': 13, 'så': 14, 'med': 15, 'har': 16, 'af': 17, 'den': 18, 'som': 19, 'kan': 20, 'du': 21, 'et': 22, 'vi': 23, 'men': 24, 'man': 25, 'om': 26, 'var': 27, 'skal': 28, 'være': 29, 'da': 30, 'han': 31, 'bare': 32, 'hvis': 33, 'hvor': 34, 'også': 35, 'her': 36, 'fra': 37, 'eller': 38, 'url': 39, 'jo': 40, 'hvad': 41, 'godt': 42, 'noget': 43, 'når': 44, 'user': 45, 'lige': 46, 'ved': 47, 'nu': 48, 'ud': 49, 'sig': 50, 'op': 51, 'dem': 52, 'alle': 53, 'danmark': 54, 'hun': 55, 'vil': 56, 'meget': 57, 'mig': 58, 'kunne': 59, 'lidt': 60, 'over': 61, 'have': 62, 'selv': 63, 'bliver': 64, 'deres': 65, 'nok': 66, 'dig': 67, 'ville': 68, 'må': 69, 'min': 70, 'se': 71, 'få': 72, 'helt': 73, 'dansk': 74, 'havde': 75, 'r': 76, 'år': 77, 'vores': 78, 'ja': 79, 'skulle': 80, 'mere': 81, 'kommer': 82, 'folk': 83, 'alt': 84, 'mange': 85, 'end': 86, 'ser': 87, 'danske': 88, 'os': 89, '

In [0]:
!pip install keras-self-attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.42.0-cp36-none-any.whl size=17296 sha256=43d492c5fbb9ec84c8811ce2f928596e2b3aa25d3c0e0538c381ec469143cfed
  Stored in directory: /root/.cache/pip/wheels/7b/05/a0/99c0cf60d383f0494e10eca2b238ea98faca9a1fe03cac2894
Successfully built keras-self-attention


In [0]:
from keras.layers import Concatenate
from sklearn.metrics import precision_recall_fscore_support

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras_self_attention import SeqSelfAttention
from keras.layers import CuDNNGRU,CuDNNLSTM,GlobalMaxPooling1D,GlobalAveragePooling1D
from sklearn.utils import class_weight
class Attention(Layer):
    def __init__(self,step_dim=20,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

#max_len=

def BidLstm(maxlen, max_features, embed_size):
    inp1 = Input(shape=(maxlen, ))
    #inp2=Input(shape=(1,))
    x1=Embedding(max_words+1,embed_size)(inp1)
    #x1 = Embedding(max_words + 1,embed_size,weights=[embedding_matrix_1],
    #              trainable=True)(inp1)
    # x2 = Embedding(len(tok.word_index) + 1,embed_size_2,weights=[embedding_matrix_2],
    #                trainable=True)(inp1)
    # x3 = Embedding(len(tok.word_index) + 1,embed_size_3,weights=[embedding_matrix_3],
    #                trainable=True)(inp1)
    # x1 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x1)
    # x2 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x2)
    # x3 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x3)   
    #x1 = Attention(maxlen)(x1)
    # x2 = Attention(maxlen)(x2)
    # x3 = Attention(maxlen)(x3)
    # x=  Concatenate()([x1,x2,x3])
    x1 = CuDNNLSTM(200, return_sequences=True)(x1)   
    x2=  GlobalMaxPooling1D()(x1)
    x3= GlobalAveragePooling1D()(x1)
    x=  Concatenate()([x2,x3])
    x = Dropout(0.1)(x)
    #x = Attention(maxlen)(x)
    # layer = Dense(600,name='FC1')(x)
    # layer = Dense(300,activation='relu')(layer)
    layer = Dense(128,activation='relu')(x)
 #   layer = BatchNormalization(name = 'BN1')(layer)
    #layer = Activation('relu')(layer)
    #layer = Dropout(0.4)(layer)
    layer = Dense(64,name='FC2')(layer)
#    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
   # layer=  Concatenate()([layer,inp2])
    # layer=Dense(256,activation='relu')(layer)
    # layer=Dense(128,activation='relu')(layer)
    layer = Dense(1,name='out_layer',activation='sigmoid')(layer)

    model = Model(inputs=[inp1],outputs=layer)

    return model
model_bi=BidLstm(max_len,max_features=max_words,embed_size=100)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model_bi.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc',km.f1_score()])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [0]:
model_bi.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 45)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 45, 100)      300100      input_1[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)        (None, 45, 200)      241600      embedding_1[0][0]                
__________________________________________________________________________________________________
global_max_pooling1d_1 (GlobalM (None, 200)          0           cu_dnnlstm_1[0][0]               
____________________________________________________________________________________________

In [0]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

In [0]:
# print(class_weights)

In [0]:
cp_filepath='/content/drive/My Drive/offenseval/'+'checkpoints/lstm_model_2020a_danish.h5'
cp_check_point=keras.callbacks.ModelCheckpoint(cp_filepath, monitor='val_f1_score', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period=1)
es = EarlyStopping(monitor='val_f1_score', mode='max', min_delta=0,patience=2,restore_best_weights=True)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [0]:
# model_bi.fit([sequences_matrix_train],y_train,validation_data=([sequences_matrix_dev],y_dev),epochs=10,batch_size=32,class_weight=class_weights,callbacks=[es,cp_check_point])

In [0]:
# y_preds_dl=model_bi.predict(sequences_matrix_dev)

In [0]:
from sklearn.metrics import classification_report

#y_pred = model_bi.predict(X_dev, batch_size=30, verbose=1)

# print(classification_report(y_dev, y_preds_dl.round()))

In [0]:
# train_data.head()

In [0]:
y=(train_data['subtask_a']=="OFF").astype(int)
# sequences_train = tok.texts_to_sequences(df_train['tweet'].astype(str))
# vocab_size = len(tok.word_index) + 1
# sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [0]:
f1s=[]
for train_ind,dev_ind in kfold.split(train_data['tweet'],y):
  #print(train_data['tweet'][train_ind].shape)
  x_train=train_data['tweet'][train_ind]
  y_train=y[train_ind]
  x_dev=train_data['tweet'][dev_ind]
  y_dev=y[dev_ind]
  seq_train=tok.texts_to_sequences(x_train.astype(str))
  seq_matrix_train=sequence.pad_sequences(seq_train,maxlen=max_len,padding='post',truncating='post')
  seq_dev=tok.texts_to_sequences(x_dev.astype(str))
  seq_matrix_dev=sequence.pad_sequences(seq_dev,maxlen=max_len,padding='post',truncating='post')
  model_bi.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc',km.f1_score()])
  model_bi.fit([seq_matrix_train],y_train,validation_split=0.1,epochs=10,batch_size=32,class_weight=class_weights,callbacks=[es,cp_check_point])
  scores = model_bi.evaluate(seq_matrix_dev, y_dev, verbose=0)
  pred = model_bi.predict(seq_matrix_dev)
  print(classification_report(y_dev, pred.round()))
  f1s.append(f1_score(y_dev,pred.round(),average='macro'))
  #f1s.append(scores[2])
  print(scores)
print("Mean f1:{0}, standard dev:{1}".format(np.mean(f1s),np.std(f1s)))

Train on 2397 samples, validate on 267 samples
Epoch 1/10
2397/2397 [==============================] - 2s 778us/step - loss: 0.0316 - acc: 0.9917 - f1_score: 0.9687 - val_loss: 3.4256 - val_acc: 0.8464 - val_f1_score: 0.3692
Epoch 2/10
2397/2397 [==============================] - 1s 306us/step - loss: 0.0299 - acc: 0.9917 - f1_score: 0.9687 - val_loss: 3.7579 - val_acc: 0.8502 - val_f1_score: 0.3750
Epoch 3/10
2397/2397 [==============================] - 1s 305us/step - loss: 0.0320 - acc: 0.9912 - f1_score: 0.9671 - val_loss: 3.2205 - val_acc: 0.8539 - val_f1_score: 0.3810
Epoch 4/10
2397/2397 [==============================] - 1s 309us/step - loss: 0.0455 - acc: 0.9862 - f1_score: 0.9490 - val_loss: 2.8864 - val_acc: 0.8577 - val_f1_score: 0.4242
Epoch 5/10
2397/2397 [==============================] - 1s 302us/step - loss: 0.0283 - acc: 0.9917 - f1_score: 0.9687 - val_loss: 3.0753 - val_acc: 0.8539 - val_f1_score: 0.4179
Epoch 6/10
2397/2397 [==============================] - 1s 317u

In [0]:
from sklearn.metrics import f1_score

In [0]:
pred = model_bi.predict(seq_matrix_dev)
print(f1_score(y_dev,pred.round(),average='macro'))
print(classification_report(y_dev, pred.round()))

0.9342660448589829
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       257
           1       0.88      0.90      0.89        39

    accuracy                           0.97       296
   macro avg       0.93      0.94      0.93       296
weighted avg       0.97      0.97      0.97       296



In [0]:
classification_report(y_dev, pred.round())

TypeError: ignored